In [1]:
import torch
import polyak


x = torch.randn(2, requires_grad=True, dtype=torch.double)
def loss_function():
    return sum(torch.abs(x))

def closure():
    optimizer.zero_grad()
    loss = loss_function()
    loss.backward()
    return loss

param = [x]
optimizer = polyak.Polyak(param)
# A short loop that applies the Polyak subgradient method to the loss function
for i in range(100):
    loss = optimizer.step(closure)
    print("Iteration: {}, Loss: {}".format(i, loss))
    print("x: {}".format(x.data))


Iteration: 0, Loss: 1.5565787483386422
x: tensor([ 0.0415, -0.0415], dtype=torch.float64)
Iteration: 1, Loss: 0.08292535629767539
x: tensor([5.5511e-17, 5.5511e-17], dtype=torch.float64)
Iteration: 2, Loss: 1.1102230246251565e-16
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 3, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 4, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 5, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 6, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 7, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 8, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 9, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 10, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 11, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 12, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration: 13, Loss: 0.0
x: tensor([0., 0.], dtype=torch.float64)
Iteration:

In [ ]:
# Quick working test of superpolyak with numpy functions
import util
import numpy as np
import torch

def f(y):
    return np.sum(np.abs(y))

def gradf(y):
    return np.sign(y)

d = 5
max_elts = d
y0 = np.random.randn(d)
while f(y0) > 1e-20:
    y, d = util.build_bundle_naive(f=f, gradf=gradf, y0=y0, tau=1.,eta_est=1.5,min_f=0., max_elts=max_elts)
    y0 = y
    print("f(y)", f(y0))

In [ ]:
# A pytorch compatible version of the above code.
import util
import numpy as np
import torch

d = 5
max_elts = d
x = torch.randn(d, requires_grad=True, dtype=torch.double)
def f():
    return torch.sum(torch.abs(x))

# Populate the gradient with something.
f().backward()
# Closure function to allow us to call backward.
def closure():
    x.grad.zero_()
    loss = f()
    loss.backward()
    return loss

current_iter = 0
while closure().item() > 1e-20 and current_iter < 100:
    y, bundle_idx = util.build_bundle_naive_torch(closure, x=x, tau=1.,eta_est=1.5,min_f=0., max_elts=max_elts)
    x.data = y
    print("f(y)", closure().item())
    print("Bundle index", bundle_idx)
    current_iter += 1